# 🎛️ Professional Mixing Session
## Multi-Channel Mixing → Mixed WAV Export

This notebook provides professional mixing of raw audio channels, creating a balanced stereo mix.

**Workflow:**
1. Load raw channels (organized by instrument groups)
2. Apply professional mixing (EQ, compression, spatial positioning, effects)
3. Export final mixed WAV file
4. Optionally export stems for further processing

In [1]:
# Core imports and setup
import os
import json
import numpy as np
import soundfile as sf
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our mixing system
from mixing_engine import MixingSession, ChannelStrip, MixBus
from channel_recognition import identify_channel_type, suggest_processing
from mix_templates import MixTemplate, get_template

print("🎛️ Professional Mixing System Ready!")
print("📁 Supports multi-channel input with intelligent processing")
print("🎚️ Exports mixed WAV file")

DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
🎛️ Mixing Engine loaded!
   • Multi-channel support with intelligent routing
   • Template-based processing
   • Bus architecture with grouping
   • Exports stems for post-mix pipeline
🔍 Channel Recognition System loaded!
   • Name pattern matching with 40+ instrument types
   • Frequency analysis for content identification
   • Automatic processing suggestions
   • User hint support for ambiguous channels
🎨 Mix Templates loaded!
   • 5 genre-specific templates (Pop, Rock, EDM, Hip-Hop, Jazz)
   • Channel-specific EQ, compression, and effects
   • Intelligent spatial po

## 📁 Step 1: Define Your Input Channels

Organize your raw audio files by instrument category.

In [2]:
# Load from folder structure
def load_from_folder_structure(base_path: str) -> Dict:
    """Load channels from organized folder structure"""
    channels = {}
    base = Path(base_path)
    
    print(f"🔍 Looking for channels in: {base_path}")
    
    if not base.exists():
        print(f"❌ Directory not found: {base_path}")
        print("📝 Please update the base_path variable below with your actual channel directory")
        return {}
    
    # Expected categories
    categories = ['drums', 'bass', 'guitars', 'keys', 'vocals', 'backvocals', 'synths', 'strings', 'brass', 'percussion', 'fx', 'other']
    
    for category in categories:
        category_path = base / category
        if category_path.exists():
            channels[category] = {}
            audio_files = list(category_path.glob('*.wav'))
            if audio_files:
                print(f"  ✅ Found {category}/ with {len(audio_files)} files")
                for audio_file in audio_files:
                    channel_name = audio_file.stem
                    channels[category][channel_name] = str(audio_file)
            else:
                print(f"  ⚠️ Found {category}/ but no .wav files")
    
    return channels

# ⚠️ UPDATE THIS PATH TO YOUR ACTUAL CHANNEL DIRECTORY! ⚠️
base_path = "/Users/itay/Documents/post_mix_data/pre_mix_channels/combined_inst/"

print("📁 IMPORTANT: Update the base_path above with your actual directory!")
print("   Expected structure: your_directory/drums/kick.wav, your_directory/vocals/lead.wav, etc.")
print()

channels = load_from_folder_structure(base_path)

# Display loaded channels
def display_channels(channels):
    print("📁 Loaded Channels:")
    total = 0
    for category, tracks in channels.items():
        if tracks:
            print(f"\n  {category.upper()}:")
            for name, path in tracks.items():
                print(f"    • {name}: {os.path.basename(path)}")
                total += 1
    
    if total == 0:
        print("\n❌ NO CHANNELS LOADED!")
        print("\n🛠️ To fix this:")
        print("   1. Update the base_path variable above with your actual directory")
        print("   2. Make sure your audio files are organized like:")
        print("      your_directory/drums/kick.wav")
        print("      your_directory/bass/bass.wav")  
        print("      your_directory/vocals/lead.wav")
        print("   3. Re-run this cell")
        print("\n⚠️ Without channels, the mixer will create empty/silent files!")
    else:
        print(f"\n✅ Ready to mix {total} channels!")
    
    return total

total_channels = display_channels(channels)

# Safety check
if total_channels == 0:
    print("\n🛑 STOP: No channels loaded - please fix the path before continuing!")

📁 IMPORTANT: Update the base_path above with your actual directory!
   Expected structure: your_directory/drums/kick.wav, your_directory/vocals/lead.wav, etc.

🔍 Looking for channels in: /Users/itay/Documents/post_mix_data/pre_mix_channels/combined_inst/
  ✅ Found drums/ with 5 files
  ✅ Found bass/ with 5 files
  ✅ Found guitars/ with 6 files
  ✅ Found keys/ with 4 files
  ✅ Found vocals/ with 3 files
  ✅ Found backvocals/ with 5 files
  ✅ Found synths/ with 3 files
  ⚠️ Found fx/ but no .wav files
📁 Loaded Channels:

  DRUMS:
    • tom: tom.wav
    • hihat: hihat.wav
    • kick: kick.wav
    • snare: snare.wav
    • cymbal: cymbal.wav

  BASS:
    • bass_guitar5: bass_guitar5.wav
    • bass1: bass1.wav
    • bass_guitar3: bass_guitar3.wav
    • bass_synth2: bass_synth2.wav
    • bass_synth4: bass_synth4.wav

  GUITARS:
    • electric_guitar4: electric_guitar4.wav
    • electric_guitar5: electric_guitar5.wav
    • electric_guitar6: electric_guitar6.wav
    • electric_guitar2: electric

## 🎚️ Step 2: Choose Mix Template

In [3]:
# Select template
selected_template = "modern_pop"  # Change this to your genre

# Optional: Customize template parameters
template_customization = {
    "brightness": 0.7,      # 0-1 (dark to bright)
    "width": 0.8,          # 0-1 (mono to wide)
    "aggression": 0.8,     # 0-1 (gentle to aggressive)
    "vintage": 0.3,        # 0-1 (modern to vintage)
    "dynamics": 0.4,       # 0-1 (compressed to dynamic)
    "depth": 0.7,          # 0-1 (flat to deep)
}

print(f"🎛️ Selected Template: {selected_template}")
print("\n📊 Template Characteristics:")
for param, value in template_customization.items():
    bar = '█' * int(value * 10) + '░' * int((1-value) * 10)
    print(f"  {param:12} [{bar}] {value:.1%}")

🎛️ Selected Template: modern_pop

📊 Template Characteristics:
  brightness   [███████░░░] 70.0%
  width        [████████░] 80.0%
  aggression   [████████░] 80.0%
  vintage      [███░░░░░░░] 30.0%
  dynamics     [████░░░░░░] 40.0%
  depth        [███████░░░] 70.0%


## 🎛️ Step 3: Configure Mix Settings

In [4]:
# 🎚️ MIX BALANCE PRESETS 🎚️
# Choose a preset or create your own custom balance

# Preset 1: Vocal-Forward (for singer-songwriter, pop vocals)
vocal_forward = {
    "vocal_prominence": 0.8,     # Prominent vocals
    "drum_punch": 0.4,           # Subtle drums
    "bass_foundation": 0.5,      # Balanced bass
    "instrument_presence": 0.3,  # Background instruments
}

# Preset 2: Drum-Heavy (for rock, metal, energetic tracks)
drum_heavy = {
    "vocal_prominence": 0.5,     # Balanced vocals
    "drum_punch": 0.9,           # Punchy, aggressive drums
    "bass_foundation": 0.7,      # Strong foundation
    "instrument_presence": 0.6,  # Present instruments
}

# Preset 3: Balanced Mix (neutral starting point)
balanced = {
    "vocal_prominence": 0.05,     # Balanced vocals
    "drum_punch": 0.8,           # Balanced drums
    "bass_foundation": 0.5,      # Balanced bass
    "instrument_presence": 0.5,  # Balanced instruments
}

# Preset 4: Your Issue Fix (loud vocals, weak drums)
fix_balance = {
    "vocal_prominence": 0.01,     # 🎤 Reduce vocal dominance
    "drum_punch": 0.99,           # 🥁 Increase drum punch
    "bass_foundation": 0.6,      # 🎸 Solid foundation
    "instrument_presence": 0.4,  # 🎹 Balanced instruments
}

# 🎯 CHOOSE YOUR BALANCE (uncomment one line)
# selected_balance = vocal_forward
# selected_balance = drum_heavy  
# selected_balance = balanced
selected_balance = fix_balance     # ← This should fix your vocals/drums issue

# Or create your own custom balance:
# selected_balance = {
#     "vocal_prominence": 0.4,    # Adjust as needed (0.0 to 1.0)
#     "drum_punch": 0.7,          # Adjust as needed (0.0 to 1.0)
#     "bass_foundation": 0.6,     # Adjust as needed (0.0 to 1.0)
#     "instrument_presence": 0.5, # Adjust as needed (0.0 to 1.0)
# }

print("🎚️ Selected Balance Profile:")
for param, value in selected_balance.items():
    param_name = param.replace('_', ' ').title()
    bar = '█' * int(value * 10) + '░' * int((1-value) * 10)
    print(f"  {param_name:18} [{bar}] {value:.1%}")

print("\n💡 How it works:")
print("  • Vocal Prominence: Adjusts presence EQ, compression, reverb send")
print("  • Drum Punch: Enhances transients, compression attack, EQ punch frequencies") 
print("  • Bass Foundation: Controls low-end EQ, compression, mud removal")
print("  • Instrument Presence: Manages clarity EQ, spatial positioning vs vocals")

🎚️ Selected Balance Profile:
  Vocal Prominence   [░░░░░░░░░] 1.0%
  Drum Punch         [█████████] 99.0%
  Bass Foundation    [██████░░░░] 60.0%
  Instrument Presence [████░░░░░░] 40.0%

💡 How it works:
  • Vocal Prominence: Adjusts presence EQ, compression, reverb send
  • Drum Punch: Enhances transients, compression attack, EQ punch frequencies
  • Bass Foundation: Controls low-end EQ, compression, mud removal
  • Instrument Presence: Manages clarity EQ, spatial positioning vs vocals


# 🎚️ MIX BALANCE PRESETS 🎚️
# Choose a preset or create your own custom balance

# Preset 1: Vocal-Forward (for singer-songwriter, pop vocals)
vocal_forward = {
    "vocal_prominence": 0.8,     # Prominent vocals
    "drum_punch": 0.4,           # Subtle drums
    "bass_foundation": 0.5,      # Balanced bass
    "instrument_presence": 0.3,  # Background instruments
}

# Preset 2: Drum-Heavy (for rock, metal, energetic tracks)
drum_heavy = {
    "vocal_prominence": 0.5,     # Balanced vocals
    "drum_punch": 0.75,          # Max safe drum punch (system caps at 0.75)
    "bass_foundation": 0.7,      # Strong foundation
    "instrument_presence": 0.6,  # Present instruments
}

# Preset 3: Balanced Mix (neutral starting point)
balanced = {
    "vocal_prominence": 0.5,     # Balanced vocals
    "drum_punch": 0.5,           # Balanced drums
    "bass_foundation": 0.5,      # Balanced bass
    "instrument_presence": 0.5,  # Balanced instruments
}

# Preset 4: Your Issue Fix - ANTI-CLIP VERSION
fix_balance = {
    "vocal_prominence": 0.3,     # 🎤 Reduce vocal dominance
    "drum_punch": 0.65,          # 🥁 Moderate drum punch (safe level)
    "bass_foundation": 0.6,      # 🎸 Solid foundation
    "instrument_presence": 0.5,  # 🎹 Balanced instruments
}

# Preset 5: Ultra-Safe (if still clipping, try this)
ultra_safe = {
    "vocal_prominence": 0.4,     # 🎤 Slightly reduce vocals
    "drum_punch": 0.55,          # 🥁 Very conservative drum boost
    "bass_foundation": 0.5,      # 🎸 Balanced foundation
    "instrument_presence": 0.5,  # 🎹 Balanced instruments
}

# Preset 6: No Drum Boost (pure EQ/compression balance, no level changes)
no_drum_boost = {
    "vocal_prominence": 0.35,    # 🎤 Reduce vocals via EQ/compression only
    "drum_punch": 0.5,           # 🥁 Neutral level, punch via EQ/compression only
    "bass_foundation": 0.5,      # 🎸 Balanced
    "instrument_presence": 0.5,  # 🎹 Balanced
}

# 🎯 CHOOSE YOUR BALANCE (uncomment one line)
# selected_balance = vocal_forward
# selected_balance = drum_heavy  
# selected_balance = balanced
selected_balance = fix_balance        # ← Anti-clip version (try this first)
# selected_balance = ultra_safe       # ← If still clipping, try this
# selected_balance = no_drum_boost    # ← If desperate - only EQ/compression, no gain

# Or create your own custom balance:
# selected_balance = {
#     "vocal_prominence": 0.4,    # Adjust as needed (0.0 to 1.0)
#     "drum_punch": 0.6,          # MAX 0.75 to prevent clipping!
#     "bass_foundation": 0.6,     # Adjust as needed (0.0 to 1.0)
#     "instrument_presence": 0.5, # Adjust as needed (0.0 to 1.0)
# }

print("🎚️ Selected Balance Profile:")
for param, value in selected_balance.items():
    param_name = param.replace('_', ' ').title()
    bar = '█' * int(value * 10) + '░' * int((1-value) * 10)
    print(f"  {param_name:18} [{bar}] {value:.1%}")

print("\n💡 How it works:")
print("  • Vocal Prominence: Adjusts presence EQ, compression, reverb send")
print("  • Drum Punch: Enhances transients, compression attack, EQ punch frequencies") 
print("  • Bass Foundation: Controls low-end EQ, compression, mud removal")
print("  • Instrument Presence: Manages clarity EQ, spatial positioning vs vocals")

print("\n🛡️ Anti-Clipping Protection:")
print("  • Drum punch automatically capped at 75% maximum")
print("  • Ultra-conservative gain adjustments (max ±0.75dB)")
print("  • Aggressive soft limiting kicks in at -3dBFS")
print("  • Minimal EQ boosts (max +1dB) to prevent buildup")
print("\n  ⚠️ If STILL clipping, try 'ultra_safe' or 'no_drum_boost' presets")

In [5]:
# 🎯 APPLY GUI BALANCE SETTINGS

# Get the balance values from the GUI sliders
if 'balance_gui' in locals():
    # Get current slider values
    gui_balance = balance_gui.get_balance_values()
    
    # Convert to the format expected by the mixing engine
    selected_balance = {
        "channel_overrides": gui_balance  # Direct channel-level control
    }
    
    print("✅ Balance settings applied from GUI:")
    
    # Show what was changed
    changed_channels = {k: v for k, v in gui_balance.items() if abs(v - 1.0) > 0.01}
    
    if changed_channels:
        print(f"\n🔧 Modified {len(changed_channels)} channels:")
        for channel_id, value in sorted(changed_channels.items()):
            change_pct = (value - 1.0) * 100
            direction = "↑" if change_pct > 0 else "↓"
            print(f"  {direction} {channel_id:40} = {value:.2f} ({change_pct:+.0f}%)")
    else:
        print("  📊 All channels at neutral balance (1.0)")
        
    print(f"\n💾 Python code for these settings:")
    print(balance_gui.get_python_code())
    
else:
    print("❌ GUI not found - please run the previous cell first")
    # Fallback to preset balance
    selected_balance = {
        "vocal_prominence": 0.3,
        "drum_punch": 0.65, 
        "bass_foundation": 0.6,
        "instrument_presence": 0.5,
    }

❌ GUI not found - please run the previous cell first


In [6]:
# 🎚️ MANUAL BALANCE CONTROL - BYPASSES BROKEN GUI

print("🎚️ MANUAL BALANCE CONTROL (BYPASSING BROKEN GUI)")
print("=" * 60)

# Direct channel overrides - no GUI needed
channel_overrides = {
    # DRUMS - Massive boost
    'drums.kick': 4.0,      # 400% boost
    'drums.snare': 4.0,     # 400% boost  
    'drums.hihat': 3.0,     # 300% boost
    'drums.tom': 3.0,       # 300% boost
    'drums.cymbal': 3.0,    # 300% boost
    
    # VOCALS - Reduce the loud ones
    'vocals.lead_vocal1': 0.4,  # 60% reduction
    'vocals.lead_vocal2': 0.4,  # 60% reduction
    'vocals.lead_vocal3': 0.4,  # 60% reduction
    
    # BASS - Slight boost
    'bass.bass_guitar5': 1.5,   # 50% boost
    'bass.bass1': 1.5,          # 50% boost
}

# Set the balance parameters
selected_balance = {
    "vocal_prominence": 0.2,    # Reduce vocals
    "drum_punch": 0.7,          # Boost drums
    "bass_foundation": 0.6,
    "instrument_presence": 0.4,
}

print(f"✅ Manual overrides set: {len(channel_overrides)} channels")
print("\n📊 DRUM BOOSTS:")
for ch, val in channel_overrides.items():
    if 'drums.' in ch:
        boost_pct = (val - 1.0) * 100
        print(f"  ↑ {ch}: {val} ({boost_pct:+.0f}%)")

print("\n📊 VOCAL REDUCTIONS:")
for ch, val in channel_overrides.items():
    if 'vocals.' in ch:
        change_pct = (val - 1.0) * 100
        print(f"  ↓ {ch}: {val} ({change_pct:+.0f}%)")

print("\n✅ Manual balance control ready!")
print("🎯 This bypasses the broken GUI completely")
print("\n🎯 Ready for mixing! Both 'selected_balance' and 'channel_overrides' are set!")

🎚️ MANUAL BALANCE CONTROL (BYPASSING BROKEN GUI)
✅ Manual overrides set: 10 channels

📊 DRUM BOOSTS:
  ↑ drums.kick: 4.0 (+300%)
  ↑ drums.snare: 4.0 (+300%)
  ↑ drums.hihat: 3.0 (+200%)
  ↑ drums.tom: 3.0 (+200%)
  ↑ drums.cymbal: 3.0 (+200%)

📊 VOCAL REDUCTIONS:
  ↓ vocals.lead_vocal1: 0.4 (-60%)
  ↓ vocals.lead_vocal2: 0.4 (-60%)
  ↓ vocals.lead_vocal3: 0.4 (-60%)

✅ Manual balance control ready!
🎯 This bypasses the broken GUI completely

🎯 Ready for mixing! Both 'selected_balance' and 'channel_overrides' are set!


## 🎚️ Step 2.6: Interactive Balance Control

Use GUI sliders to adjust balance with both group controls and individual channel precision.

In [7]:
# Check if we have channels before proceeding
if not channels or sum(len(tracks) for tracks in channels.values()) == 0:
    print("❌ Cannot proceed - no channels loaded!")
    print("📝 Please go back to Step 1 and update your channel directory path.")
else:
    # Initialize mixing session
    session = MixingSession(
        channels=channels,
        template=selected_template,
        template_params=template_customization,
        sample_rate=44100,
        bit_depth=24
    )
    
    # Configure mix settings with intelligent balance
    mix_settings = {
        "buses": {
            "drum_bus": {"channels": ["drums.*"], "compression": 0.8, "glue": 0.4},
            "bass_bus": {"channels": ["bass.*"], "compression": 0.8, "saturation": 0.2},
            "vocal_bus": {"channels": ["vocals.*", "backvocals.*"], "compression": 0.3, "presence": 0.5},
            "instrument_bus": {"channels": ["guitars.*", "keys.*", "synths.*"], "width": 0.7},
        },
        "sends": {},  # Simplified - no effects sends
        "master": {
            "eq_mode": "gentle",
            "compression": 0.2,
            "limiter": True,
            "target_lufs": -14,
        },
        "automation": {
            "vocal_rides": False,
            "drum_fills": False,
            "outro_fade": False,
        },
        "mix_balance": selected_balance  # 🎚️ Apply chosen balance profile
    }
    
    # Add GUI channel overrides if they exist
    if 'channel_overrides' in locals() and channel_overrides:
        mix_settings["channel_overrides"] = channel_overrides
        print(f"🎚️ GUI channel overrides will be applied to mixing engine!")
    
    session.configure(mix_settings)
    print("✅ Mix configured with intelligent balance controls")

📁 Loading audio channels...
  ✓ Loaded: drums.tom
  ✓ Loaded: drums.hihat
  ✓ Loaded: drums.kick
  ✓ Loaded: drums.snare
  ✓ Loaded: drums.cymbal
  ✓ Loaded: bass.bass_guitar5
  ✓ Loaded: bass.bass1
  ✓ Loaded: bass.bass_guitar3
  ✓ Loaded: bass.bass_synth2
  ✓ Loaded: bass.bass_synth4
  ✓ Loaded: guitars.electric_guitar4
  ✓ Loaded: guitars.electric_guitar5
  ✓ Loaded: guitars.electric_guitar6
  ✓ Loaded: guitars.electric_guitar2
  ✓ Loaded: guitars.acoustic_guitar1
  ✓ Loaded: guitars.electric_guitar3
  ✓ Loaded: keys.bell3
  ✓ Loaded: keys.clavinet1
  ✓ Loaded: keys.piano4
  ✓ Loaded: keys.piano2
  ✓ Loaded: vocals.lead_vocal3
  ✓ Loaded: vocals.lead_vocal2
  ✓ Loaded: vocals.lead_vocal1
  ✓ Loaded: backvocals.lead_vocal3
  ✓ Loaded: backvocals.lead_vocal2
  ✓ Loaded: backvocals.backing_vocal
  ✓ Loaded: backvocals.lead_vocal1
  ✓ Loaded: backvocals.lead_vocal4
  ✓ Loaded: synths.rythmic_synth1
  ✓ Loaded: synths.pad3
  ✓ Loaded: synths.pad2
🎚️ GUI channel overrides will be applied 

## 🎚️ Step 4: Process and Export Mix

In [8]:
# Check if session was created successfully
if 'session' not in locals():
    print("❌ Cannot process mix - no session created!")
    print("📝 Please ensure channels are loaded in Step 1 and session is configured in Step 3.")
else:
    # Create output directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f"/Users/itay/Documents/post_mix_data/mixing_sessions/session_{timestamp}"
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"📁 Output directory: {output_dir}")
    print("\n🎛️ Processing mix...\n")
    
    # Process the mix - ONLY export full mix
    try:
        mix_results = session.process_mix(
            output_dir=output_dir,
            export_individual_channels=False,
            export_buses=False,
            export_stems=False,
            export_full_mix=True,  # Only export the final mix
            progress_callback=lambda msg: print(f"  {msg}")
        )
        
        print("\n✅ Mix processing complete!")
        print("\n📊 Mix Results:")
        print(f"  • Peak Level: {mix_results['peak_db']:.1f} dBFS")
        print(f"  • RMS Level: {mix_results['rms_db']:.1f} dBFS")
        print(f"  • LUFS: {mix_results['lufs']:.1f}")
        print(f"  • Dynamic Range: {mix_results['dynamic_range']:.1f} dB")
        print(f"  • Processing Time: {mix_results['time']:.1f} seconds")
        
        # Find and display the exported mix file
        mix_file = os.path.join(output_dir, "full_mix.wav")
        if os.path.exists(mix_file):
            size_mb = os.path.getsize(mix_file) / (1024 * 1024)
            if size_mb > 0.01:  # At least 10KB
                print(f"\n📁 Final Mix: {mix_file} ({size_mb:.1f} MB)")
                print("✅ Your mix is ready!")
            else:
                print(f"\n⚠️ Mix file created but is very small ({size_mb:.3f} MB)")
                print("   This usually means no input channels were processed.")
        else:
            print("\n⚠️ Mix file not found")
            
    except Exception as e:
        print(f"\n❌ Error during mixing: {e}")
        import traceback
        traceback.print_exc()

📁 Output directory: /Users/itay/Documents/post_mix_data/mixing_sessions/session_20250829_011818

🎛️ Processing mix...

  Processing individual channels...
  Processing drum_bus...
⚠️ Applied clipping protection to tom (peak was 3.8dBFS, reduced to -3.7dBFS)
⚠️ Applied clipping protection to hihat (peak was 6.1dBFS, reduced to -3.7dBFS)


KeyboardInterrupt: 

In [ ]:
# 🔍 DEBUG: Check if drums are actually boosted after processing
print("🔍 DEBUGGING DRUM LEVELS AFTER PROCESSING")
print("=" * 50)

if 'session' in locals():
    print("📊 Checking individual channel gains after all processing:")
    for channel_id in ["drums.kick", "drums.snare", "drums.hihat", "drums.tom", "drums.cymbal"]:
        if channel_id in session.channel_strips:
            strip = session.channel_strips[channel_id]
            print(f"  {channel_id}: gain = {strip.gain:.3f}")
        else:
            print(f"  ❌ {channel_id} not found")
    
    print("\n📊 Checking bus levels:")
    if "drum_bus" in session.buses:
        drum_bus = session.buses["drum_bus"]
        print(f"  drum_bus: gain = {getattr(drum_bus, 'gain', 'N/A')}")
    
    print("\n🎛️ Testing individual drum channels (before bus processing):")
    # Let's process just the kick drum individually to see if it's loud
    if "drums.kick" in session.channel_strips:
        kick_strip = session.channel_strips["drums.kick"]
        print(f"  Kick drum gain: {kick_strip.gain:.3f}")
        
        # Check if kick has audio
        kick_audio = kick_strip.audio
        if kick_audio is not None and len(kick_audio) > 0:
            import numpy as np
            kick_peak = np.max(np.abs(kick_audio))
            kick_rms = np.sqrt(np.mean(kick_audio**2))
            print(f"  Kick raw audio peak: {kick_peak:.3f}")
            print(f"  Kick raw audio RMS: {kick_rms:.3f}")
            
            # Apply gain and see what we get
            boosted_audio = kick_audio * kick_strip.gain
            boosted_peak = np.max(np.abs(boosted_audio))
            boosted_rms = np.sqrt(np.mean(boosted_audio**2))
            print(f"  Kick after 2x boost - peak: {boosted_peak:.3f}, RMS: {boosted_rms:.3f}")
            
            if boosted_peak > 0.1:
                print("  ✅ Kick should be audible after boost")
            else:
                print("  ❌ Kick still very quiet even after boost - source audio might be silent")
        else:
            print("  ❌ Kick has no audio data")
    
else:
    print("❌ No session found")

In [ ]:
# 🔍 MIXING ENGINE DIAGNOSTIC - WHY NO DRUM POWER & BAD MIX QUALITY

print("🔍 DIAGNOSING MIXING ENGINE PROBLEMS")
print("=" * 60)
print("Checking why drums have no power and mix sounds horrible...")

if 'session' in locals():
    import numpy as np
    
    # 1. Check if GUI changes are actually applied
    print("\n1️⃣ GUI vs ENGINE COMPARISON:")
    if 'channel_overrides' in locals() and channel_overrides:
        print(f"   ✅ GUI provided {len(channel_overrides)} channel overrides")
        for ch, gui_val in list(channel_overrides.items())[:5]:
            if ch in session.channel_strips:
                engine_gain = session.channel_strips[ch].gain
                match = "✅" if abs(gui_val - engine_gain) < 0.1 else "❌"
                print(f"   {match} {ch}: GUI={gui_val:.2f} vs Engine={engine_gain:.2f}")
            else:
                print(f"   ❌ {ch}: GUI={gui_val:.2f} vs Engine=NOT_FOUND")
    else:
        print("   ❌ No GUI overrides found - sliders not working!")
    
    # 2. Check bus compression effects (MAIN SUSPECT)
    print("\n2️⃣ BUS COMPRESSION ANALYSIS:")
    buses = session.mix_settings.get('buses', {})
    for bus_name, bus_config in buses.items():
        compression = bus_config.get('compression', 0)
        if compression > 0.5:
            print(f"   ⚠️ {bus_name}: HEAVY compression = {compression} (may crush dynamics)")
        else:
            print(f"   ✅ {bus_name}: compression = {compression}")
    
    # 3. Master processing check (SECOND SUSPECT)  
    print("\n3️⃣ MASTER PROCESSING:")
    master_config = session.mix_settings.get('master', {})
    master_comp = master_config.get('compression', 0)
    target_lufs = master_config.get('target_lufs', -14)
    limiter = master_config.get('limiter', False)
    
    print(f"   Master compression: {master_comp}")
    print(f"   Target LUFS: {target_lufs} (louder = more crushing)")
    print(f"   Limiter enabled: {limiter}")
    
    if master_comp > 0.1:
        print("   ⚠️ Master compression may be squashing everything")
    if target_lufs > -16:
        print("   ⚠️ Target LUFS too loud - causes over-limiting")
    
    # 4. Drum power analysis
    print("\n4️⃣ DRUM POWER ANALYSIS:")
    drum_channels = [ch for ch in session.channel_strips.keys() if 'drums.' in ch]
    
    total_drum_contribution = 0
    for ch in drum_channels:
        if ch in session.channel_strips:
            strip = session.channel_strips[ch]
            if strip.audio is not None:
                boosted_peak = np.max(np.abs(strip.audio * strip.gain))
                total_drum_contribution += boosted_peak
                boost_db = 20 * np.log10(strip.gain) if strip.gain > 0 else -60
                print(f"   🥁 {ch}: gain={strip.gain:.2f} ({boost_db:+.1f}dB), peak={boosted_peak:.3f}")
    
    print(f"   📊 Total drum contribution: {total_drum_contribution:.3f}")
    
    if total_drum_contribution < 0.5:
        print("   ❌ DRUMS TOO QUIET - even after boosting!")
        print("      Problem likely: Bus compression crushing boosted drums")
    else:
        print("   ✅ Drums should be loud enough")
    
    # 5. Mix quality assessment
    print("\n5️⃣ MIX QUALITY PROBLEMS:")
    
    problems = []
    
    # Check for over-processing
    total_compression = sum(bus.get('compression', 0) for bus in buses.values()) + master_comp
    if total_compression > 2.0:
        problems.append(f"Too much compression: {total_compression:.1f} total")
    
    # Check for limiting issues
    if target_lufs > -16 and limiter:
        problems.append(f"Over-limiting: {target_lufs}dB target with limiter")
    
    # Check template settings
    if 'template_customization' in globals():
        aggression = template_customization.get('aggression', 0)
        if aggression > 0.7:
            problems.append(f"Template too aggressive: {aggression}")
    
    if problems:
        for problem in problems:
            print(f"   ❌ {problem}")
    else:
        print("   🤔 No obvious processing problems found")
    
    # 6. SOLUTIONS
    print("\n6️⃣ IMMEDIATE SOLUTIONS TO TRY:")
    print("=" * 40)
    
    print("\n   🥁 FOR DRUM POWER:")
    print("      1. Reduce drum_bus compression: 0.8 → 0.2")
    print("      2. Increase drum gains in GUI to 3.0+") 
    print("      3. OR disable drum bus entirely")
    
    print("\n   🎚️ FOR MIX QUALITY:")
    print("      1. Reduce master compression: 0.2 → 0.05")
    print("      2. Change target LUFS: -14 → -18")
    print("      3. Reduce ALL bus compression by 70%")
    print("      4. Set template aggression to 0.3 (from 0.8)")

else:
    print("❌ No session found")

In [ ]:
# 🔍 COMPREHENSIVE AUDIO QUALITY ANALYSIS - ALL 31 CHANNELS

print("🔍 COMPREHENSIVE SOURCE AUDIO ANALYSIS")
print("=" * 70)
print("Analyzing ALL 31 audio channels for quality issues...")

if 'session' in locals():
    import numpy as np
    
    # Categories to organize results
    categories = {}
    problem_files = []
    good_files = []
    
    print("\nFormat: Channel Name → Peak | RMS | Status")
    print("-" * 70)
    
    # Analyze all channels
    for channel_id, strip in session.channel_strips.items():
        category = channel_id.split('.')[0]
        track_name = channel_id.split('.')[1]
        
        if category not in categories:
            categories[category] = []
        
        # Analyze audio quality
        if strip.audio is not None and len(strip.audio) > 0:
            peak = np.max(np.abs(strip.audio))
            rms = np.sqrt(np.mean(strip.audio**2))
            
            # Quality assessment
            if peak > 0.7:
                quality = "✅ GOOD"
                good_files.append(channel_id)
            elif peak > 0.3:
                quality = "⚠️ QUIET"
                problem_files.append({'channel': channel_id, 'peak': peak, 'issue': 'QUIET'})
            elif peak > 0.1:
                quality = "❌ VERY QUIET"
                problem_files.append({'channel': channel_id, 'peak': peak, 'issue': 'VERY QUIET'})
            else:
                quality = "🔇 NEARLY SILENT"
                problem_files.append({'channel': channel_id, 'peak': peak, 'issue': 'NEARLY SILENT'})
            
            categories[category].append({
                'name': track_name,
                'peak': peak,
                'rms': rms,
                'quality': quality,
                'gain': strip.gain
            })
        else:
            quality = "💀 NO AUDIO"
            problem_files.append({'channel': channel_id, 'peak': 0, 'issue': 'NO AUDIO'})
            categories[category].append({
                'name': track_name,
                'peak': 0,
                'rms': 0,
                'quality': quality,
                'gain': strip.gain
            })
    
    # Display by category
    for category, tracks in categories.items():
        print(f"\n📁 {category.upper()}:")
        for track in tracks:
            peak_str = f"{track['peak']:.3f}"
            rms_str = f"{track['rms']:.3f}" 
            gain_str = f"gain:{track['gain']:.2f}"
            print(f"  {track['name']:18} → {peak_str:>6} | {rms_str:>6} | {track['quality']} ({gain_str})")
    
    # Summary Analysis
    print("\n" + "="*70)
    print("📊 COMPREHENSIVE ANALYSIS SUMMARY")
    print("="*70)
    
    total_files = len(problem_files) + len(good_files)
    problem_pct = (len(problem_files) / total_files * 100) if total_files > 0 else 0
    
    if problem_files:
        print(f"\n❌ PROBLEM FILES ({len(problem_files)} of {total_files} channels = {problem_pct:.0f}%):")
        print("These files need level correction:")
        
        # Group problems by severity
        nearly_silent = [p for p in problem_files if p['peak'] < 0.05]
        very_quiet = [p for p in problem_files if 0.05 <= p['peak'] < 0.1]
        quiet = [p for p in problem_files if 0.1 <= p['peak'] < 0.3]
        
        if nearly_silent:
            print(f"\n  🔇 NEARLY SILENT/EMPTY ({len(nearly_silent)} files):")
            for prob in sorted(nearly_silent, key=lambda x: x['peak']):
                print(f"    • {prob['channel']:35} (peak: {prob['peak']:.3f})")
        
        if very_quiet:
            print(f"\n  ❌ VERY QUIET ({len(very_quiet)} files):")
            for prob in sorted(very_quiet, key=lambda x: x['peak']):
                print(f"    • {prob['channel']:35} (peak: {prob['peak']:.3f})")
        
        if quiet:
            print(f"\n  ⚠️ QUIET ({len(quiet)} files):")
            for prob in sorted(quiet, key=lambda x: x['peak']):
                print(f"    • {prob['channel']:35} (peak: {prob['peak']:.3f})")
    
    if good_files:
        print(f"\n✅ GOOD FILES ({len(good_files)} channels):")
        print("These files have proper audio levels (peak > 0.7):")
        for good in sorted(good_files)[:8]:
            print(f"  • {good}")
        if len(good_files) > 8:
            print(f"  • ... and {len(good_files) - 8} more")
    
    # Final recommendations
    print(f"\n🎯 RECOMMENDATIONS FOR SOURCE MATERIAL:")
    if problem_pct > 50:
        print(f"  🚨 MAJOR ISSUE: {problem_pct:.0f}% of your source files are too quiet!")
        print(f"  📋 Tell your source provider:")
        print(f"     'Most audio files have very low levels (peaks under 0.3)'")
        print(f"     'Please normalize all files to -6dB peak level'") 
        print(f"     'Files should have peaks between 0.7-0.9, not 0.05-0.3'")
    elif problem_pct > 25:
        print(f"  ⚠️ MODERATE ISSUE: {problem_pct:.0f}% of files need level correction")
        print(f"  📋 Tell your source provider:")
        print(f"     'Some audio files are too quiet - normalize to -6dB peak'")
    else:
        print(f"  ✅ GOOD: Most files ({100-problem_pct:.0f}%) have proper levels")

else:
    print("❌ No session found")

## 💾 Optional: Save Session Data

In [ ]:
# Save session information for reference  
if 'session' in locals() and 'mix_results' in locals():
    try:
        # Convert numpy values to Python types for JSON serialization
        def convert_for_json(obj):
            if isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            return obj
        
        # Clean the results for JSON serialization
        clean_results = {}
        for key, value in mix_results.items():
            clean_results[key] = convert_for_json(value)
        
        session_data = {
            "timestamp": timestamp,
            "template": selected_template,
            "template_params": template_customization,
            "mix_settings": mix_settings,
            "results": clean_results,
            "output_file": os.path.join(output_dir, "full_mix.wav"),
            "channels_processed": len([c for cat in channels.values() for c in cat.keys()])
        }
        
        session_file = os.path.join(output_dir, "mix_session.json")
        with open(session_file, 'w') as f:
            json.dump(session_data, f, indent=2)
        
        print(f"💾 Session saved: {session_file}")
        print("\n✅ Mix complete! Your final mix is ready.")
        
    except Exception as e:
        print(f"⚠️ Could not save session data: {e}")
else:
    print("⚠️ No session or results to save - please run the mixing process first.")

## 📈 Optional: Export Stems Too

Run this cell if you also want to export stems for further processing:

In [ ]:
# Export stems for further processing (optional)
if 'session' in locals():
    try:
        print("📤 Exporting stems...")
        
        stem_export_config = {
            "format": "wav",
            "bit_depth": 24,
            "sample_rate": 44100,
            "normalization": "peak",
            "target_level": -6.0,
        }
        
        stem_mapping = {
            "drums": ["drum_bus"],
            "bass": ["bass_bus"],
            "vocals": ["vocal_bus"],
            "music": ["instrument_bus"],
        }
        
        exported_stems = session.export_stems(
            output_dir=os.path.join(output_dir, "stems"),
            stem_mapping=stem_mapping,
            config=stem_export_config
        )
        
        print("✅ Stems exported:")
        for stem_name, stem_path in exported_stems.items():
            if os.path.exists(stem_path):
                size_mb = os.path.getsize(stem_path) / (1024 * 1024)
                print(f"  • {stem_name}: {os.path.basename(stem_path)} ({size_mb:.1f} MB)")
            else:
                print(f"  • {stem_name}: File not created")
                
    except Exception as e:
        print(f"❌ Error exporting stems: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ No session available - please run the mixing process first.")